In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

In [2]:
K = 4
lambda_k_true = np.array([3,4,-1,1.5])

I = 40
J = 50


sigma = 1
S = 100
### generate exogenous data
np.random.seed(10877)

# φ_i_j_k = np.random.normal(1,1, size=[I,J,K-1])
φ_i_j_k = (np.random.normal(0,1,size = [I])[:,None,None] 
            * np.random.normal(1,1, size=[J,K-1])[None,:,:])



eps_si_j = sigma * np.random.normal(0,1, size = [S*I,J])


def φ_k(i,B):
    # return φ_i_j_k[i,B,:].sum(0)
    return np.concatenate((φ_i_j_k[i,B,:].sum(0), [ - np.sum(B)**(1.5)]))

### greedy
def greedy(i,lambda_k, p_j = np.zeros(J)):
    B_k =  np.zeros(J, dtype=bool)
    val = np.inner(φ_k(i,B_k),lambda_k)
    k = 0 
    while k < J:
        j_k = None
        max_add_j =  - np.inf
        for j in np.where(1-B_k)[0]:
            val_add_j = np.inner(φ_k(i,B_k+ np.eye(1,J,j,dtype=bool)[0]), lambda_k)   - p_j[j] - p_j[B_k].sum() - val
            if val_add_j > max_add_j:
                j_k = j
                max_add_j = val_add_j
            
        if j_k is None or max_add_j < 0:
            break
        val += max_add_j 
        B_k[j_k] = 1
        k += 1
    return B_k , val

In [3]:
greedy(0,lambda_k_true)

(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False,  True,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False]),
 19.286843517697946)

$\begin{align}
\min_{p}& \quad \frac{1}{S}\sum_{si} u_{si} + \sum_{j} p_j \notag\\
s.t.& \quad u_{si} + \sum_{j \in B}p_j \geq \phi_{iB}^\top\lambda + \epsilon_{siB}\quad \text{ for all }siB \notag
\end{align}$

In [4]:
max_iters = 100
tol = 1e-12

constraints_list = []

# Create the environment with license parameters
with gp.Env(params=options) as env:
    # Create a Gurobi model within the environment
    with gp.Model(env=env) as model:
        ### Initialize 
        # Create variables
        u_si = model.addVars(S*I, name="utilities")
        # p_j = model.addVars(J, name="prices")
        p_j = model.addVars(J, lb= 0 ,ub=GRB.INFINITY, name="prices")

        # Set objective
        model.setObjective((1/S)* u_si.sum() + p_j.sum(), GRB.MINIMIZE)
        
        # Optimize the model
        model.setParam('OutputFlag', 0)
        model.optimize()
        
        # Extract the solution (theta_solution)
        theta_solution = np.array(model.x)

        
        ### Column Generation
        iter = 0
        while iter < max_iters:
            print(f"ITER: {iter}")
            ### Pricing problem
            B_star_si = []
            val_si = []
            for si in range(S*I):
                        B_star, val = greedy(si //S ,lambda_k = lambda_k_true, p_j = theta_solution[-J:]- eps_si_j[si,:])
                        B_star_si.append(B_star)
                        val_si.append(val -  theta_solution[si])
            # stop if certificate holds
            print(f"reduced cost: {np.max(val_si)}")
            if np.max(val_si) <= tol:
                primal_solution = np.array(model.x)
                dual_solution = np.array(model.pi)
                print("DONE!")
                break
            
            ### Master problem
            model.addConstrs((u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])>= 
                            φ_k(si // S, B_star_si[si])@lambda_k_true + eps_si_j[si, B_star_si[si]].sum()
                            for si in range(S*I)), name="constraint_batch")
            
            constraints_list.append(B_star_si)

            # Optimize the model
            u_si.start = val_si + theta_solution[:S*I]
            p_j.start = theta_solution[-J:]
            model.optimize()
            theta_solution = np.array(model.x)
            iter += 1
            print("##############")
            

Set parameter WLSAccessID
Set parameter WLSSecret


Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
ITER: 0
reduced cost: 329.9763281144321
##############
ITER: 1
reduced cost: 150.25069142022443
##############
ITER: 2
reduced cost: 120.4337429708522
##############
ITER: 3
reduced cost: 104.30462989149567
##############
ITER: 4
reduced cost: 65.70046563790345
##############
ITER: 5
reduced cost: 90.12688426722504
##############
ITER: 6
reduced cost: 55.54514180233417
##############
ITER: 7
reduced cost: 25.36709827516931
##############
ITER: 8
reduced cost: 37.01218655855628
##############
ITER: 9
reduced cost: 31.05478070354786
##############
ITER: 10
reduced cost: 14.620783134504443
##############
ITER: 11
reduced cost: 11.541384601807941
##############
ITER: 12
reduced cost: 8.897716362360843
##############
ITER: 13
reduced cost: 4.565327525531066
##############
ITER: 14
reduced cost: 3.3614679366684186
##############
ITER: 15
reduced cost: 1.63087761986744

In [5]:
binding_constraints = np.array(constraints_list).reshape(-1,J)[dual_solution >0]
index = np.kron(np.ones(iter, dtype= int),np.arange(S*I))
matching = np.zeros((I * S,J), dtype=bool)
matching[index[dual_solution > 0]]= binding_constraints

In [6]:
print(matching.sum(0))
print(matching.sum(1))

[100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100]
[0 0 0 ... 0 1 0]


In [68]:
np.random.seed(9989877)
I = 200
random_draws = np.random.choice(np.arange(np.shape(matching)[0]), size= [I], replace=False)
sample = matching[random_draws,:]
draw_type = random_draws//S
m_j = sample.sum(0)
phi_k_hat = np.zeros(K)
for i in range(I):
    phi_k_hat += φ_k(draw_type[i],sample[i,:])
print(phi_k_hat)
print(m_j)
print(draw_type)

[ 350.66634463  420.23864576  190.85247425 -503.90795476]
[8 5 3 8 4 6 3 3 8 4 5 4 4 5 3 4 8 4 8 5 5 3 5 5 3 3 5 5 8 6 6 6 7 7 5 5 5
 8 7 2 8 5 7 4 5 7 4 5 3 3]
[ 2 34 30  4 22  8 35 20 11 22  0 35 26 38 27 33 17 25 13 21 18 22 15 19
  0  4  0 11 19 20 39 28 12 27 37 23 39 19 25 16 20 21 10 21 32  7 26 25
 19  6  0  2 17 36 16 16 10 10 15 22 39 34  7 31 29  0  9 28 13 13 10 19
 14 35 15  8 17 21 14  9 26 25  6  7 32 37 31 30  7 24 19 28  5  0 27  2
 13 35 30 30  5 10 11 28  0 37  7 11 33 21 24 18 26 16 11 16  4  4  0 18
 35  0  7 23 11  9 27 31 35 19 35 22  2  7 17  0 29 22 34  7 30  7 10 36
 11 21  2 35 21  9 21 19 37 35  1 29 25 26 32 23 17  7 15 23  2 15 21 15
 21 16 27 17 27 36  9 23 24  5  1 14 29 14  3  5  4 27 21 17  4 15 37 35
 30 29 27 18  6 14 30  2]


$\begin{align}
\max_{\lambda ,u,p}& \quad \sum_{ik} \phi_{i\hat B_i,k} \lambda_k - \frac{1}{S}\sum_{si} u_{si} -  \sum_{j} p_j \notag\\
s.t.& \quad u_{si} + \sum_{j \in B}p_j \geq \sum_{k} \phi_{i B,k} \lambda_k + \epsilon_{siB}\quad \text{ for all }siB \notag
\end{align}$

In [70]:
max_iters = 100
tol = 1e-12

np.random.seed(443)
S = 50
eps_si_j = sigma * np.random.normal(0,1, size = [S*I,J])      

with gp.Env(params=options) as env:
    with gp.Model(env=env) as model:
        ### Initialize 
        # Create variables
        lambda_k = model.addVars(K, lb= -GRB.INFINITY, ub = GRB.INFINITY , name="parameters")
        u_si = model.addVars(S*I, name="utilities")
        # p_j = model.addVars(J, name="prices")
        p_j = model.addVars(J,  lb= -GRB.INFINITY, ub = GRB.INFINITY , name="prices")

        # Set objective
        model.setObjective(gp.quicksum(phi_k_hat[k] * lambda_k[k] for k in range(K))
                            - (1/S)* u_si.sum() -  gp.quicksum(m_j[j] * p_j[j] for j in range(J)), GRB.MAXIMIZE)
        
        model.addConstrs((u_si[si] + gp.quicksum(p_j[j] for j in np.where(sample[si // S])[0])>= 
                            gp.quicksum(φ_k(draw_type[si // S], sample[si // S])[k] *  lambda_k[k] for k in range(K))
                            + eps_si_j[si, sample[si // S]].sum()
                            for si in range(S*I)), name="constraint_batch")
        
        # Optimize the model
        model.setParam('OutputFlag', 0)
        # model.setParam('Presolve', 0)
        model.optimize()
        status = model.Status
        # Extract the solution 
        theta_solution = np.array(model.x)

        ### Column Generation
        iter = 0
        while iter < max_iters:
            print(f"ITER: {iter}, parameters: {theta_solution[:K]}")
            ### Pricing problem
            B_star_si = []
            val_si = []
            for si in range(S*I):
                        B_star, val = greedy(draw_type[si //S] , lambda_k = theta_solution[:K], p_j = theta_solution[-J:]- eps_si_j[si,:])
                        B_star_si.append(B_star)
                        val_si.append(val -  theta_solution[K+si])
            print(f"greedy done")

            # stop if certificate holds
            print(f"reduced cost: {np.max(val_si)}")
            if np.max(val_si) <= tol:
                primal_solutin = np.array(model.x)
                dual_solution = np.array(model.pi)
                break
            
            ### Master problem
            # add constraints
            model.addConstrs((u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])>= 
                            gp.quicksum(φ_k(draw_type[si // S], B_star_si[si])[k] *  lambda_k[k] for k in range(K))+ 
                            eps_si_j[si, B_star_si[si]].sum()
                            for si in range(S*I)), name="constraint_batch")
            
            # optimize the model
            lambda_k.start = theta_solution[:K]
            u_si.start = val_si + theta_solution[K:S*I +K]
            p_j.start = theta_solution[-J:]
            model.optimize()
            theta_solution = np.array(model.x)
            iter += 1
            print("##############")
            

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
ITER: 0, parameters: [ 0.57446244  0.90569662 -0.00546338  0.588401  ]
greedy done
reduced cost: 71.09929507374856
##############
ITER: 1, parameters: [ 3.47256465  3.81954549 -0.73370001  2.5748386 ]
greedy done
reduced cost: 124.98878787145142
##############
ITER: 2, parameters: [ 78.02411415  97.70262689 -14.29152444  78.09590042]
greedy done
reduced cost: 3347.611416852711
##############
ITER: 3, parameters: [10.59768967 12.07187936 -2.42377588 25.37545818]
greedy done
reduced cost: 1337.6190709219004
##############
ITER: 4, parameters: [ 70.2703299   91.55672331 -13.74430952  71.17418367]
greedy done
reduced cost: 2271.8010444775778
##############
ITER: 5, parameters: [ 0.57329983 -0.0209916  -0.4781127   5.95955691]
greedy done
reduced cost: 320.879645335825
##############
ITER: 6, parameters: [ 0.57329983 

In [53]:
print(lambda_k_true)
print(theta_solution[:K])

[ 3.   4.  -1.   1.5]
[ 2.03431249  2.99351178 -0.65004187  1.10330597]
